In [16]:
# load tidy dataset

import pandas as pd
import os.path
import numpy as np
from itertools import combinations
import glob
import sys
import time

#import numpy as np
#import loadTidyCSV.py

def loadTidy(tidyData): 
    assert os.path.isfile(tidyData), "desired file does not exist" 
    df = pd.read_csv(tidyData, header = None, names = ["date", "animal", "session", "trial", "stimulus", "neuronID", "timePt", "CaSignal"])
    return df 

# helper function to return the number of neurons in the provided data frame
getNumNeurons = lambda df: len(np.unique(df['neuronID'].tolist()))
getNeurons = lambda df: np.unique(df['neuronID'].tolist())

# pass in pre-filtered data set containing data for only one animal and the same session (ie SAME NEURONS!)
def getListsOfTrialIDs(df_animalSession):
    ## get trials for both stimuli
    df_anmlSessStimA = df_animalSession[df_animalSession['stimulus'] == stimA]
    df_anmlSessStimB = df_animalSession[df_animalSession['stimulus'] == stimB]
    print(stimA,stimB)

    ## get lists of trial numbers of each stimuli's presentations 
    trials_stimA = np.unique(df_anmlSessStimA['trial'].tolist())
    trials_stimB = np.unique(df_anmlSessStimB['trial'].tolist())
    print("trial IDs for each stimulus type",trials_stimA,trials_stimB)
    return (trials_stimA,trials_stimB)

## pass in a data frame with only a single animal and session    
def getNumTimePtsPerTrial(df_animalSession, trials_stimA, trials_stimB):

    #### get number of timePts in each trial selected above 
    ## (1 to 3 presentations of the same stimuli exist per session in Prabhat's data)
    numTimePtsPerTrial = np.empty((2,max(len(trials_stimA),len(trials_stimB))))
    numTimePtsPerTrial[:] = np.nan
    stimInd = 0;
    for thisStimTypeTrialNums in [trials_stimA, trials_stimB]:
        trialInd = 0
        for trial in thisStimTypeTrialNums:
            inds_thisTrial = (df_animalSession['trial']==trial)
            tmp_df_thisTrial = df_animalSession[inds_thisTrial] # gives all time points for all neurons
            numNeurons = getNumNeurons(tmp_df_thisTrial) 
            numTimePtsPerTrial[stimInd,trialInd] = np.sum(inds_thisTrial)/numNeurons 
            trialInd += 1
        stimInd += 1
    print(numTimePtsPerTrial) # rows are for stimuli type; cols are presentation of that stimulus
    return numTimePtsPerTrial

## test candidate comparisons based on whether the number of trials per session and approximate number of timePts match
def areNumTrialsPerStimulusEqual(numTimePtsPerTrial):
    
    ## no trials of either type --> discard this comparison for this animal/session   
    if np.all(np.isnan(numTimePtsPerTrial)):
        print("DISCARDED: neither stimulus type were found for this animal and session")
        return False  # skip to next session (WORK: handle this)
        
    ## different numbers of trials per stimuli/session --> discard this comparison for this animal/session 
    elif np.any(np.isnan(numTimePtsPerTrial)): 
        print("DISCARDED: mismatching numbers of trials per stimulus type for this animal/session")
        return False # skip to next session (WORK: handle this)

    ## FULFILLED here: condition that allows analysis to proceed to attempted data
    elif not np.any(np.isnan(numTimePtsPerTrial)): 
        print("trial numbers match")
    else:
        raise RuntimeError('unexpected trial comparison occurred')
        return False
    
    print("checking approx num of time points")

## input argument generated from getNumTimePtsPerTrial
def areNumTimePtsPerTrialSimilar(numTimePtsPerTrial):
    minTPs, maxTPs, meanTPs, stdTPs = timePtStats(numTimePtsPerTrial)             
    if (np.abs(minTPs-meanTPs) > (threshTPs_stdFromMean * np.abs(meanTPs-stdTPs))) \
        or (np.abs(maxTPs-meanTPs) > (threshTPs_stdFromMean * np.abs(meanTPs-stdTPs))): 
        print("DISCARDED: variance in trial length is above the user's threshold")
        return False # skip to next session (WORK: handle this)

    ### passed all criteria if it made it this far
    return True

## input argument created by getNumTimePtsPerTrial function
def timePtStats(numTimePtsPerTrial):
    minTPs = int(np.amin(numTimePtsPerTrial))
    maxTPs = int(np.amax(numTimePtsPerTrial))
    meanTPs = np.mean(numTimePtsPerTrial)
    stdTPs = np.std(numTimePtsPerTrial)
    print('min', minTPs)
    print('max', maxTPs)
    print('std', stdTPs)
    print('mean',meanTPs)
    print('|min-mean|=',np.abs(minTPs-meanTPs))
    print('|max-mean|=',np.abs(maxTPs-meanTPs))
    print('|mean-std|=',np.abs(meanTPs-stdTPs))
    print('thresh * |mean-std|=',(threshTPs_stdFromMean * np.abs(meanTPs-stdTPs)))
    return minTPs, maxTPs, meanTPs, stdTPs

In [2]:
################ concatenate all .csv files exported from matlab into single pandas dataframe df

# tidy csv file and dir (use makeTidy_Anderson.m to convert Ann's structure to csv)
tidyDataDir = '/home/orthogonull/a_MHR/a_research/a_gitResearch/git_ignored/imagingAnalysis/data/2_tidyFormat/'
tidyDataFileTemplate = 'mouse'
tidyDataFileExt = '.csv'

# get all input files you want to add to the same dataset
dataFiles = np.sort(glob.glob(\
    "/home/orthogonull/a_MHR/a_research/a_gitResearch/git_ignored/imagingAnalysis/data/2_tidyFormat/mouse*.csv"))
print("data files: \n", dataFiles)


print("\n loading and appending to prior pandas data frame")
dataLst = []
for file in dataFiles:
    print(file)
    dataLst.append(loadTidy(file))
df = pd.concat(dataLst)

print('finished loading')

############# ALL DATA STORED HERE IN DF
    

data files: 
 [ '/home/orthogonull/a_MHR/a_research/a_gitResearch/git_ignored/imagingAnalysis/data/2_tidyFormat/mouse1.csv'
 '/home/orthogonull/a_MHR/a_research/a_gitResearch/git_ignored/imagingAnalysis/data/2_tidyFormat/mouse2.csv'
 '/home/orthogonull/a_MHR/a_research/a_gitResearch/git_ignored/imagingAnalysis/data/2_tidyFormat/mouse3.csv'
 '/home/orthogonull/a_MHR/a_research/a_gitResearch/git_ignored/imagingAnalysis/data/2_tidyFormat/mouse4.csv'
 '/home/orthogonull/a_MHR/a_research/a_gitResearch/git_ignored/imagingAnalysis/data/2_tidyFormat/mouse5.csv']

 loading and appending to prior pandas data frame
/home/orthogonull/a_MHR/a_research/a_gitResearch/git_ignored/imagingAnalysis/data/2_tidyFormat/mouse1.csv
/home/orthogonull/a_MHR/a_research/a_gitResearch/git_ignored/imagingAnalysis/data/2_tidyFormat/mouse2.csv
/home/orthogonull/a_MHR/a_research/a_gitResearch/git_ignored/imagingAnalysis/data/2_tidyFormat/mouse3.csv
/home/orthogonull/a_MHR/a_research/a_gitResearch/git_ignored/imagingAn

In [3]:
## survey/search data to prepare for split operation
metaStrs = [['dates','date'],['animals','animal'],['sessions','session'],['maxTrials','trial'],['stimuli','stimulus']]

print("searching over entire data set to get range of various IDs for data (used in subsequent loops)") 

## this dictionary holds useful info regarding the range of inputs to loop/search over subsequently
metaDct = {}
for a,b in metaStrs:
    print(a,b)
    metaDct[a] = np.unique(df[b].tolist())
print(metaDct)


searching over entire data set to get range of various IDs for data (used in subsequent loops)
dates date
animals animal
sessions session
maxTrials trial
stimuli stimulus
{'dates': array(['2017_05_00'], 
      dtype='<U10'), 'animals': array([1, 3, 4, 5, 7]), 'sessions': array([1, 2, 3]), 'maxTrials': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21]), 'stimuli': array(['USS', 'baseline', 'female', 'male', 'mineral oil odor',
       'peanut odor', 'pred odor', 'rat', 'tone', 'toy'], 
      dtype='<U16')}


In [4]:
######### USER PARAMETERS #########
threshTPs_stdFromMean = 1

In [19]:
# get all pairs of stimuli
stimCmbTpl = tuple(combinations(metaDct['stimuli'],2)) 

######### MAIN LOOP ##########
totalNumComparisons = 0;
for (stimA, stimB) in stimCmbTpl:
    print((stimA,stimB))
    totalNumComparisons += 1
    print("stimuli comparison num: ", totalNumComparisons)
    
    ## get all data for both trial types
    indsBoth = (df['stimulus']==stimA) | (df['stimulus']==stimB)
    df_bothStimuli = df[indsBoth]

    #### select data by animals and sessions
    for animal in metaDct['animals']:
        print('animal: ', animal)
        for session in metaDct['sessions']:
            print('session:', session)
            
            ## return subselection of data where the same neurons were recorded
            inds_animalSession = (df_bothStimuli['animal'] == animal) & (df_bothStimuli['session'] == session)
            df_animalSession = df_bothStimuli[inds_animalSession]
            
            # get lists of trial IDs matching stimuli
            trials_stimA, trials_stimB = getListsOfTrialIDs(df_animalSession)
            
            #### skip this comparison if the data don't match in number of trials or approx number of timePts
            numTimePtsPerTrial = getNumTimePtsPerTrial(df_animalSession,trials_stimA,trials_stimB)
            if areNumTrialsPerStimulusEqual(numTimePtsPerTrial)==False:
                break            
            
            ## truncate longer trials to shortest trial and save to new df
            minTPs, maxTPs, meanTPs, stdTPs = timePtStats(numTimePtsPerTrial)
            truncLst = []
            for trial in np.concatenate((trials_stimA,trials_stimB)):
                tmp_inds_trunc = (df_animalSession['trial']==trial) & (df_animalSession['timePt'] < minTPs)
                tmp_df_trunc = df_animalSession[tmp_inds_trunc]
                truncLst.append(tmp_df_trunc)
            df_trunc = pd.concat(truncLst)
                    
            ## concatenate same cells 
            NUM_ROWS_INITIALIZE = 15 # this value shouldn't matter since it should expand if needed 
            neuronArr_anmlSess = np.empty((NUM_ROWS_INITIALIZE,minTPs-1)) # -1 for 0 indexing
            trials = np.unique(df_trunc['trial'].tolist())
            for stimLst in [trials_stimA, trials_stimB]:
                for trial in stimLst:
                    neurons = getNeurons(df_trunc)
                    print("appending same neurons in trial: ", trial)
                    tmp_sameStimNeuronsArr = np.empty((len(neurons),minTPs-1)) # for 0 indexing
                    
                    ## this loop probably isn't necessary!!!!!!!!!!!!!!!
                    for neuron in neurons:
                        tmp_neuronInds = (df_trunc['trial']==trial) & (df_trunc['neuronID']==neuron)
                        tmp_neuronSeries = df_trunc.loc[tmp_neuronInds,'CaSignal']
                        tmp_neuronVec = tmp_neuronSeries.as_matrix()
                        tmp_sameStimNeuronsArr[neuron-1,:] = tmp_neuronVec
                    
                    neuronArr_anmlSess
           
                
            print('\n')
        print('########\n')

print('total number of comparisons: ', totalNumComparisons)

('USS', 'baseline')
stimuli comparison num:  1
animal:  1
session: 1
USS baseline
trial IDs for each stimulus type [ 5 10 16] [ 1  8 15]
[[ 1117.  1116.  1117.]
 [  370.   363.  1476.]]
trial numbers match
checking approx num of time points
min 363
max 1476
std 415.865663406
mean 926.5
|min-mean|= 563.5
|max-mean|= 549.5
|mean-std|= 510.634336594
thresh * |mean-std|= 510.634336594
appending same neurons in trial:  5
appending same neurons in trial:  10
appending same neurons in trial:  16
appending same neurons in trial:  1
appending same neurons in trial:  8
appending same neurons in trial:  15


session: 2
USS baseline
trial IDs for each stimulus type [ 4 11 21] [ 1  7 15]
[[ 1015.  1329.  1214.]
 [  689.   681.   685.]]
trial numbers match
checking approx num of time points
min 681
max 1329
std 266.773155821
mean 935.5
|min-mean|= 254.5
|max-mean|= 393.5
|mean-std|= 668.726844179
thresh * |mean-std|= 668.726844179
appending same neurons in trial:  4
appending same neurons in trial: 

animal:  1
session: 1
USS male
trial IDs for each stimulus type [ 5 10 16] [ 6 13 17]
[[ 1117.  1116.  1117.]
 [ 1118.  1125.  1116.]]
trial numbers match
checking approx num of time points
min 1116
max 1125
std 3.13138237134
mean 1118.16666667
|min-mean|= 2.16666666667
|max-mean|= 6.83333333333
|mean-std|= 1115.0352843
thresh * |mean-std|= 1115.0352843
appending same neurons in trial:  5
appending same neurons in trial:  10
appending same neurons in trial:  16
appending same neurons in trial:  6
appending same neurons in trial:  13
appending same neurons in trial:  17


session: 2
USS male
trial IDs for each stimulus type [ 4 11 21] [ 2 13 18]
[[ 1015.  1329.  1214.]
 [ 1166.  1113.  1167.]]
trial numbers match
checking approx num of time points
min 1015
max 1329
std 95.2832036032
mean 1167.33333333
|min-mean|= 152.333333333
|max-mean|= 161.666666667
|mean-std|= 1072.05012973
thresh * |mean-std|= 1072.05012973
appending same neurons in trial:  4
appending same neurons in trial:  11
ap

appending same neurons in trial:  11
appending same neurons in trial:  3
appending same neurons in trial:  12


session: 2
USS mineral oil odor
trial IDs for each stimulus type [5 9] [ 6 10]
[[ 1133.  1124.]
 [ 1125.  1116.]]
trial numbers match
checking approx num of time points
min 1116
max 1133
std 6.0207972894
mean 1124.5
|min-mean|= 8.5
|max-mean|= 8.5
|mean-std|= 1118.47920271
thresh * |mean-std|= 1118.47920271
appending same neurons in trial:  5
appending same neurons in trial:  9
appending same neurons in trial:  6
appending same neurons in trial:  10


session: 3
USS mineral oil odor
trial IDs for each stimulus type [ 6 10] [ 5 16]
[[ 1122.  1116.]
 [ 1118.  1115.]]
trial numbers match
checking approx num of time points
min 1115
max 1122
std 2.68095132369
mean 1117.75
|min-mean|= 2.75
|max-mean|= 4.25
|mean-std|= 1115.06904868
thresh * |mean-std|= 1115.06904868
appending same neurons in trial:  6
appending same neurons in trial:  10
appending same neurons in trial:  5
appendin

KeyboardInterrupt: 

In [ ]:
#                     tmp_trialInds = (data['trial']==trial) 
#                     print(np.sum(tmp_trialInds))
#                     tmp_mat = data[tmp_trialInds].as_matrix()
#                     print(tmp_mat)
#                     print(type(tmp_neuronConcatArr))
#                     np.append(tmp_neuronConcatArr,tmp_mat)
#                     print(tmp_neuronConcatArr)

In [ ]:
testInds = (df['animal']==1) & (df['session']==1) & (df['stimulus']=='USS') & (df['trial']==5)
print(np.sum(testInds))

In [ ]:
np.unique(df_anmlSessStimA[df_anmlSessStimA['trial'] == trialA].tolist())

In [ ]:

# gb = df.groupby(['stimulusType']).get_group('rat')

stimGrouped = df.groupby('stimulusType')
ratGrpd = stimGrouped.get_group('rat')

print(ratGrpd)

print('reached end of file')

gb = df.groupby('stimulusType')

df_means = gb.apply(np.mean)
df_means

for stimType in gb

In [ ]:
ratData = df.loc[df['stimulusType'] == 'rat',:]
ussData = df.loc[df['stimulusType'] == 'USS',:]

print(ussData)

# slicing
df_big_force = df.loc[df['impact force (mN)'] > 1000, :]

df = pd.concat((df_low, df_high), axis=1)

# Specify indices we want (note parentheses holding each Boolean)
inds = (df['food density'] == 'high') & (df['cross-sectional area (sq micron)'] > 2000)

# Pull out areas
df.loc[inds, 'cross-sectional area (sq micron)']

df.corr()


# Rename the impact force column
df = df.rename(columns={'impact force (mN)': 'impf'})



# Write out DataFrame
df.to_csv('xa_combined.csv', index=False)